In [2]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 6.8 MB/s eta 0:00:00


In [5]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.0


In [6]:
pip install tensorflow

  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)


In [3]:
# Including required python libraries used in this project
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

In [4]:
# reading train emoji and test emoji data which are attached in the repo
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [5]:
# Checking data by showing first 5 rows of the train data
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [6]:
# Checking data by showing first 5 rows of the test data
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [7]:
# Creating dictionary for some emoji's, consisting of key - number and value - emoji
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [9]:
# Printing each emoji icon by emojizing each emoji
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print(emoji.emojize(emoji_dict[ix]))

0 :heart:
1 ⚾
2 :smile:
3 :disappointed:
4 🍴


In [10]:
# Creating training and testing data
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [14]:
# Assuming train and test are DataFrames
X_train = train[0].values.tolist()
X_test = test[0].values.tolist()

# Split the data into words
X_train = [' '.join(x) for x in X_train]
X_train = [x.split() for x in X_train]

X_test = [' '.join(x) for x in X_test]
X_test = [x.split() for x in X_test]

In [16]:
# Convert labels to categorical form
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train)

In [17]:
# Now checking the above conversion by printing train and test data at 0th index
print (X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [18]:
# To check maximum length of sentence in training data
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [19]:
# To check maximum length of senetence in testing data
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [21]:
import nltk
nltk.download('glove6B50d')

[nltk_data] Error loading glove6B50d: Package 'glove6B50d' not found
[nltk_data]     in index


False

In [25]:
# Creating  embeddings dictionary with key = word and value = list of words in glove vector
embeddings_index = {}

f = open('/content/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [26]:
# Checking length of a particular word
embeddings_index["i"].shape

(50,)

In [27]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.3109366892692532

In [28]:
# Checking cosine similarity of words India and Delhi
spatial.distance.cosine(embeddings_index["india"], embeddings_index["delhi"])

0.18572333169701127

In [29]:
# Checking cosine similarity of words france and paris
spatial.distance.cosine(embeddings_index["france"], embeddings_index["paris"])

0.19746710054573613

In [36]:
# Get the maximum length of sequences
max_length = max(len(x) for x in X_train)

# Create embedding matrices with shape (num_samples, max_length, 50)
embedding_matrix_train = np.zeros((len(X_train), max_length, 50))
embedding_matrix_test = np.zeros((len(X_test), max_length, 50))

for ix in range(len(X_train)):
    for ij in range(len(X_train[ix])):
        if X_train[ix][ij].lower() in embeddings_index:
            embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]

for ix in range(len(X_test)):
    for ij in range(len(X_test[ix])):
        if X_test[ix][ij].lower() in embeddings_index:
            embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [37]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


# Training model

## - Using RNN

In [46]:
# A simple RNN network to classify the emoji class from an input Sentence

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_4 (SimpleRNN)             │ (None, 10, 64)              │           7,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_5 (SimpleRNN)             │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             325 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 5)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,941 (62.27 KB)

 Trainable params: 15,941 (62.27 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# Setting Loss and Optimiser for the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
# Training of the model and Setting hyperparameters for the model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.1714 - loss: 1.9003
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2837 - loss: 1.6607 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1591 - loss: 1.8589  
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2370 - loss: 1.7062  
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2801 - loss: 1.6790 
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3608 - loss: 1.6159 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3810 - loss: 1.5217 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3903 - loss: 1.4015  
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4066 - loss: 1.3693  
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3927 - loss: 1.3669 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5126 - loss: 1.2751 
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6379 - loss: 1.119

In [50]:
# Prediction of the trained model
pred = model.predict(embedding_matrix_test)
pred_classes = np.argmax(pred, axis=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


### ACCURACY

In [56]:
# Calculating the accuracy of the algorithm
accuracy = float(sum(pred_classes == Y_test)) / len(Y_test)

In [58]:
accuracy

0.6071428571428571

In [59]:
# Print the incorrectly classified samples
for ix in range(embedding_matrix_test.shape[0]):
    if pred_classes[ix] != Y_test[ix]:
        print(ix)
        print(test[0][ix], end=" ")

6
['I', 'am', 'upset'] 7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 12
['This', 'girl', 'is', 'messing', 'with', 'me'] 13
['are', 'you', 'serious', 'ha', 'ha'] 16
['work', 'is', 'horrible'] 17
['Congratulation', 'for', 'having', 'a', 'baby'] 18
['stop', 'messing', 'around'] 21
['you', 'brighten', 'my', 'day'] 23
['she', 'is', 'a', 'bully'] 25
['I', 'am', 'upset'] 26
['I', 'worked', 'during', 'my', 'birthday'] 27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 28
['enjoy', 'your', 'break'] 29
['valentine', 'day', 'is', 'near'] 35
['he', 'can', 'pitch', 'really', 'well'] 37
['I', 'am', 'starving'] 41
['I', 'like', 'your', 'jacket'] 46
['What', 'you', 'did', 'was', 'awesome'] 47
['ha', 'ha', 'ha', 'lol'] 48
['I', 'want', 'to', 'joke'] 51
['family', 'is', 'all', 'I', 'have'] 55
['I', 'did', 'not', 'have', 'breakfast'] 

In [60]:
# Predicting for our random sentence
x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]

In [62]:
# Get the class predictions
pred = model.predict(x_)
pred_classes = np.argmax(pred, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


##  - Using LSTM

In [63]:
# A simple LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 128)             │          91,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             645 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 5)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,877 (874.52 KB)

 Trainable params: 223,877 (874.52 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
# Setting Loss ,Optimiser for model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [65]:
# Training model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.2353 - loss: 1.6154
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2853 - loss: 1.5274
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3521 - loss: 1.5132
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4610 - loss: 1.4159
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4989 - loss: 1.3285
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5608 - loss: 1.2879
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6355 - loss: 1.1178
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6321 - loss: 0.9629
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6450 - loss: 0.9617
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6861 - loss: 0.8521
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7393 - loss: 0.6929
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7420 - loss: 0.7260
E

In [67]:
# Prediction of trained model
pred = model.predict(embedding_matrix_test)
pred_classes = np.argmax(pred, axis=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


### ACCURACY

In [69]:
# Calculating the accuracy of the algorithm
accuracy = float(sum(pred_classes == Y_test)) / len(Y_test)

In [70]:
accuracy

0.6071428571428571

In [72]:
# Print the incorrectly classified samples
for ix in range(embedding_matrix_test.shape[0]):
    if pred_classes[ix] != Y_test[ix]:
        print(ix)
        print(test[0][ix], end=" ")

5
['he', 'is', 'a', 'good', 'friend'] 6
['I', 'am', 'upset'] 7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 11
['work', 'is', 'hard'] 12
['This', 'girl', 'is', 'messing', 'with', 'me'] 13
['are', 'you', 'serious', 'ha', 'ha'] 16
['work', 'is', 'horrible'] 17
['Congratulation', 'for', 'having', 'a', 'baby'] 21
['you', 'brighten', 'my', 'day'] 22
['I', 'boiled', 'rice'] 23
['she', 'is', 'a', 'bully'] 26
['I', 'worked', 'during', 'my', 'birthday'] 28
['enjoy', 'your', 'break'] 29
['valentine', 'day', 'is', 'near'] 32
['My', 'life', 'is', 'so', 'boring'] 34
['will', 'you', 'be', 'my', 'valentine'] 37
['I', 'am', 'starving'] 40
['I', 'will', 'go', 'dance'] 41
['I', 'like', 'your', 'jacket'] 45
['I', 'love', 'to', 'the', 'stars', 'and', 'back'] 46
['What', 'you', 'did', 'was', 'awesome'] 55
['I', 'did', 'not', 'have', 'breakfast'] 